# GENG5551 Prototype Model Training 

Run code below if using google collab for training. If not skip to next section.

run container with:
sudo docker run --gpus all -it --rm -v $(pwd):/tf/notebooks -p 8888:8888 tensorflow/tensorflow:2.12.0-gpu-jupyter

In [ ]:
startup = False

In [ ]:
# GB: Useless

# !pip install pandas
# !pip install scikit-learn
# !pip install seaborn
# !pip install akida==2.7.2
# !pip install cnn2snn==2.7.2
# %pip install akida-models==1.5.0
# !pip install tensorflow-addons

In [ ]:
# GB: Same here
# !unzip notebooks/archive

## Dataset Preprocessing

Start here if running on anaconda virtual environment

- This section involves examining and pre-processing the data to make it suitable for use with the Akida models

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import os, shutil, random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cnn2snn
import akida as ak
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy, CategoricalCrossentropy 
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from akida_models import fetch_file, akidanet_imagenet, mobilenet_imagenet
from keras import Model
from keras.layers import Activation, Dropout, Reshape
from akida_models.layer_blocks import dense_block, conv_block
import re
from tensorflow.keras.layers import GlobalAveragePooling2D
os.chdir('notebooks/Documents/GitHub/GENG5551-Akida-Chip')

In [ ]:
os.getcwd()
# Change Akida version
os.environ["CNN2SNN_TARGET_AKIDA_VERSION"] = "v1"
# Double-check Avida version
print(' Akida version: ', cnn2snn.get_akida_version())

In [ ]:
# Load the metadata
metadata = pd.read_csv('archive/HAM10000_metadata.csv')

In [ ]:
# Define paths for train and test datasets
train_dir = 'archive/data/train'
test_dir = 'archive/data/test'

# Define the target directories for cancerous and benign images
train_cancerous_dir = 'archive/data/train/Cancerous'
train_benign_dir = 'archive/data/train/Benign'
test_cancerous_dir = 'archive/data/test/Cancerous'
test_benign_dir = 'archive/data/test/Benign'

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    channel_shift_range=50.0
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=100,
    class_mode='binary',
)

test_datagen = ImageDataGenerator()
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=100,
    class_mode='binary'
)

IMG_SIZE = 224
CLASSES = 2


# Create a base model without top layers
base_model = akidanet_imagenet(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                            classes=CLASSES,
                            alpha=0.5,
                            include_top=False,
                            pooling='None')

base_model.load_weights(f'pretrained_weights/akidanet_imagenet_224_alpha_50.h5', by_name=True)

x = base_model.output
x = conv_block(x,
               filters=64,
               kernel_size=(3,3),
               add_batchnorm=True,
               relu_activation='ReLU6',
               name='extra_conv')

# Now apply global average pooling
x = GlobalAveragePooling2D()(x)
x = dense_block(x,
                units=512,
                name='fc1',
                add_batchnorm=True,
                relu_activation='ReLU6')
x = Dropout(0.5, name='dropout_1')(x)
x = dense_block(x,
                units=CLASSES,
                name='predictions',
                add_batchnorm=False,
                relu_activation=False)

x = Reshape((CLASSES,), name='reshape1')(x)

# Build the model
model_keras = Model(base_model.input, x, name='akidanet_derma')

initial_learning_rate = 1e-3
final_learning_rate = 1e-5
decay_steps = 10
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps,
    decay_rate=(final_learning_rate / initial_learning_rate) ** (1 / decay_steps),
    staircase=True)


loss = BinaryCrossentropy()


optimizer = RMSprop(
    learning_rate=1e-3,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-7
)

# Compiling the model
model_keras.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy'])


# Setting up callbacks for saving the model and early stopping
checkpoint_cb = ModelCheckpoint(
    'akidanet_derma_best.h5', save_best_only=True)
early_stopping_cb = EarlyStopping(
    patience=10, restore_best_weights=True)

# GB: To change nb of epochs 
# (on CPU, 1 epoch lasts 22 min, so 10 is 4 hours long)
EPOCHS = 15 # Initial value: 10

# Training the model
history = model_keras.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=test_gen,
    callbacks=[checkpoint_cb, early_stopping_cb])

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history_fine_tune.history['accuracy'], label='Training Accuracy')
plt.plot(history_fine_tune.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_fine_tune.history['loss'], label='Training Loss')
plt.plot(history_fine_tune.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

mod1_fname = f"new_model.h5"
model_keras.save(mod1_fname)

tf.keras.backend.clear_session()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Obtain predictions for the training and validation datasets
train_predictions = np.argmax(model_keras.predict(train_gen), axis=-1)
val_predictions = np.argmax(model_keras.predict(test_gen), axis=-1)

# Get true labels from the generators
train_labels = train_gen.classes
val_labels = test_gen.classes

# Compute confusion matrices
train_cm = confusion_matrix(train_labels, train_predictions)
val_cm = confusion_matrix(val_labels, val_predictions)

# Plot confusion matrix for training data
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
ConfusionMatrixDisplay(confusion_matrix=train_cm).plot(cmap='Blues', ax=plt.gca())
plt.title('Confusion Matrix for Training Data')

# Plot confusion matrix for validation data
plt.subplot(1, 2, 2)
ConfusionMatrixDisplay(confusion_matrix=val_cm).plot(cmap='Blues', ax=plt.gca())
plt.title('Confusion Matrix for Validation Data')

plt.tight_layout()
plt.show()

In [27]:
batch_sizes = [32, 64, 100]
class_modes = ['binary', 'categorical', 'sparse']
base_model_origin = os.listdir('pretrained_weights')
optimizers = []

In [28]:
radam_optimizer = tfa.optimizers.RectifiedAdam(
    learning_rate=1e-3,  # Start with your current learning rate
    total_steps=10000,  # Total number of training steps
    warmup_proportion=0.1,  # 10% of the steps are used for learning rate warmup
    min_lr=1e-5  # The minimum learning rate after decay
)

optimizer = RMSprop(
    learning_rate=1e-3,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-7
)

optimizers.append(radam_optimizer)
optimizers.append(optimizer)

In [32]:
for batch_size in batch_sizes:
    for class_mode in class_modes:
        for base_model_path in base_model_origin:
            for optimizer in optimizers:
                # Construct train datagen
                train_datagen = ImageDataGenerator(
                    rotation_range=20,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    brightness_range=[0.8, 1.2],
                    channel_shift_range=50.0
                )

                train_gen = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(224, 224),
                    batch_size=batch_size,
                    class_mode=class_mode,
                )

                test_datagen = ImageDataGenerator()
                test_gen = test_datagen.flow_from_directory(
                    test_dir,
                    target_size=(224, 224),
                    batch_size=100,
                    class_mode=class_mode
                )

                IMG_SIZE = 224
                CLASSES = 2


                # Create a base model without top layers
                base_model = akidanet_imagenet(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                            classes=CLASSES,
                                            alpha=1.0,
                                            include_top=False,
                                            pooling='avg')

                base_model.load_weights(f'pretrained_weights/{base_model_path}', by_name=True)
                print("Hello")
                # layer freezing
                for layer in base_model.layers:
                    layer.trainable = False

                x = base_model.output
                x = dense_block(x,
                                units=512,
                                name='fc1',
                                add_batchnorm=True,
                                relu_activation='ReLU6')
                x = Dropout(0.5, name='dropout_1')(x)
                x = dense_block(x,
                                units=CLASSES,
                                name='predictions',
                                add_batchnorm=False,
                                relu_activation=False)

                x = Reshape((CLASSES,), name='reshape1')(x)

                # Build the model
                model_keras = Model(base_model.input, x, name='akidanet_derma')

                initial_learning_rate = 1e-3
                final_learning_rate = 1e-5
                decay_steps = 10
                lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                    initial_learning_rate,
                    decay_steps,
                    decay_rate=(final_learning_rate / initial_learning_rate) ** (1 / decay_steps),
                    staircase=True)
                

                if class_mode == 'binary':
                    loss = BinaryCrossentropy()
                elif class_mode == 'categorical':
                    loss = CategoricalCrossentropy()
                elif class_mode == 'sparse':
                    loss = SparseCategoricalCrossentropy()

                # Compiling the model
                model_keras.compile(
                    optimizer=optimizer,
                    loss=loss,
                    metrics=['accuracy'])
                

                # Setting up callbacks for saving the model and early stopping
                checkpoint_cb = ModelCheckpoint(
                    'akidanet_derma_best.h5', save_best_only=True)
                early_stopping_cb = EarlyStopping(
                    patience=10, restore_best_weights=True)

                # GB: To change nb of epochs 
                # (on CPU, 1 epoch lasts 22 min, so 10 is 4 hours long)
                EPOCHS = 15 # Initial value: 10

                # Training the model
                history = model_keras.fit(
                    train_gen,
                    epochs=EPOCHS,
                    validation_data=test_gen,
                    callbacks=[checkpoint_cb, early_stopping_cb])
                
                number = re.search(r'(\d+)(?=\.h5)', base_model_path).group()
                
                if isinstance(optimizer, tfa.optimizers.RectifiedAdam):
                    op = "Radam"
                elif isinstance(optimizer, RMSprop):
                    op = "RMSprop"


                mod1_fname = f"freeze_{batch_size}_{class_mode}_{number}_{op}.h5"
                model_keras.save(mod1_fname)

                # Fine-tuning frozen layers
                for layer in base_model.layers[-5:]:
                    layer.trainable = True

                fine_tuning_lr = 1e-4
                model_keras.compile(
                    optimizer=optimizer,
                    loss=loss,
                    metrics=['accuracy']
                )

                EPOCHS_FINE_TUNE = 10

                history_fine_tune = model_keras.fit(
                    train_gen,
                    epochs=EPOCHS_FINE_TUNE,
                    validation_data=test_gen,
                    callbacks=[checkpoint_cb, early_stopping_cb]
                )

                mod2_fname = f"unfreeze_{batch_size}_{class_mode}_{number}_{op}.h5"
                model_keras.save(mod2_fname)

                tf.keras.backend.clear_session()


                


Found 38704 images belonging to 2 classes.
Found 1002 images belonging to 2 classes.


ValueError: Shape mismatch in layer #2 (named conv_0) for weight conv_0/kernel:0. Weight expects shape (3, 3, 3, 32). Received saved weight with shape (16, 3, 3, 3)